In [8]:
import numpy as np
import utils
from utils import loadmat
from extractDigitFeatures import extractDigitFeatures
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
from keras import regularizers
from keras.constraints import unit_norm
from sklearn.model_selection import StratifiedKFold

In [9]:
def model_extractor(activation_func='relu', weight_decay=1e-4):
    # Creating an AlexNet Classifier
    model = Sequential()

    #Instantiating Layer 1
    model.add(Conv2D(48, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='valid', 
                    kernel_constraint=unit_norm(), kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

    # #Instantiating Layer 2
    model.add(Conv2D(96, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())
    # #Instantiating Layer 3
    model.add(Conv2D(192, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same',
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())

    # #Instantiating Layer 4
    model.add(Conv2D(192, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())

    # #Instantiating Layer 5
    model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())
    model.add(Flatten())

    #Instantiating Layer 6
    model.add(Dense(512, activation=activation_func)) 

    # #Instantiating Layer 8
    model.add(Dense(256, activation=activation_func))

    #Output Layer
    model.add(Dense(10, activation='softmax'))
    return model

In [12]:
def return_datasets(filename):
    data = utils.loadmat('../data/{}'.format(filename))
    trainSet, testSet, valSet = 1, 2, 3
    
    x_train = reformat(data['x'][:, :, data['set']==trainSet ])
    y_train = (data['y'][data['set']==trainSet])
    x_val = reformat(data['x'][:, :, data['set']==valSet])
    y_val = (data['y'][data['set']==valSet])
    x_test = reformat(data['x'][:, :, data['set']==testSet])
    y_test = (data['y'][data['set']==testSet])
    X = np.vstack([x_train, x_val])
    Y = np.vstack([y_train[:, np.newaxis], y_val[:, np.newaxis]])
    
    return (X, Y, x_test, y_test)

In [13]:
def reformat(x):
    new_x = np.zeros((x.shape[2], x.shape[0], x.shape[1]))
    for i in range(x.shape[2]):
        new_x[i, :] = x[:, :, i]
    return new_x[:, :, :, np.newaxis]

filename = 'digits-normal.mat'
data = utils.loadmat('../data/{}'.format(filename))
trainSet, testSet, valSet = 1, 2, 3

x_train = reformat(data['x'][:, :, data['set']==trainSet ])
y_train = (data['y'][data['set']==trainSet])
x_val = reformat(data['x'][:, :, data['set']==valSet])
y_val = (data['y'][data['set']==valSet])
x_test = reformat(data['x'][:, :, data['set']==testSet])
y_test = (data['y'][data['set']==testSet])

In [14]:
files = [ 'digits-normal.mat'] #'digits-jitter.mat', 'digits-normal.mat',
for file in files:
    X, Y, x_test, y_test = return_datasets(file)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    best_model, best_accuracy = -1, 0
    for train, test in kfold.split(X, Y):
        model = model_extractor('relu')
        model.compile(loss="categorical_crossentropy", optimizer='Adam', metrics=["accuracy"])
        model.fit(X[train], Y[train], epochs=15, batch_size=200, verbose=0)
        scores = model.evaluate(X[test], Y[test], batch_size=int(Y[test].size/5), verbose=0)
        print ("Accuracy = {}".format(scores[1]))
        if scores[1] > best_accuracy:
            best_accuracy = scores[1]
            best_model = model

        if best_accuracy > .99:
            break    
    
    print ("Test Set Accuracy for {} = {}".format(file, best_model.evaluate(x_test, y_test, verbose=0)))

Accuracy = 0.30000001192092896
Accuracy = 0.0
Accuracy = 0.0
Accuracy = 0.0
Accuracy = 0.0
Accuracy = 0.0
Accuracy = 1.0
Test Set Accuracy for digits-normal.mat = [90010.33515958977, 1.0]


In [5]:
model = model_extractor('relu')
model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(x_val, y_val), metrics=["accuracy"])
model.fit(x_train, y_train, epochs=15, batch_size=200)
#accuracy = model.evaluate(X[test], Y[test], batch_size=int(Y[test].size/5), verbose=0)[1]

Train on 1000 samples
Epoch 1/15
1000/1000 [==============================] - 3s 3ms/sample - loss: 118.0106 - accuracy: 0.2490
Epoch 2/15
1000/1000 [==============================] - 1s 641us/sample - loss: 223.0190 - accuracy: 0.0250
Epoch 3/15
1000/1000 [==============================] - 1s 641us/sample - loss: 583.8148 - accuracy: 0.0010
Epoch 4/15
1000/1000 [==============================] - 1s 646us/sample - loss: 1363.4847 - accuracy: 0.0000e+00
Epoch 5/15
1000/1000 [==============================] - 1s 641us/sample - loss: 2660.8882 - accuracy: 0.0000e+00
Epoch 6/15
1000/1000 [==============================] - 1s 715us/sample - loss: 4616.6646 - accuracy: 0.0000e+00
Epoch 7/15
1000/1000 [==============================] - 1s 937us/sample - loss: 7352.0074 - accuracy: 0.0000e+00
Epoch 8/15
1000/1000 [==============================] - 1s 1ms/sample - loss: 10976.8391 - accuracy: 0.0000e+00
Epoch 9/15
1000/1000 [==============================] - 1s 1ms/sample - loss: 15628.9297 - a

In [6]:
%%bash 
ls ../data/

denoising
digits-jitter.mat
digits-normal.mat
digits-scaled.mat
